In [2]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os

In [24]:
raw_image_directory = 'C:/Users/User/Desktop/thoracic_seg/raw_images/'
segmentation_directory = 'C:/Users/User/Desktop/thoracic_seg/segmentations/'

raw_images = sorted(os.listdir(raw_image_directory))
segmentations = sorted(os.listdir(segmentation_directory))

image_dataset = []
mask_dataset = []
image_names = []

for i, image_name in enumerate(raw_images):    
    if (image_name.split('.')[1] == 'nii'):
        image = nib.load(raw_image_directory+image_name)
        image = np.array(image.get_fdata())
        image_dataset.append(np.array(image))
        image_names.append(image_name.split('.')[0])

for i, image_name in enumerate(segmentations):    
    if (image_name.split('.')[1] == 'nii'):
        image = nib.load(segmentation_directory+image_name)
        image = np.array(image.get_fdata())
        mask_dataset.append(np.array(image))

In [45]:
num_classes = 7
stats = {}

for i in range(len(image_dataset)):
    slice = image_dataset[i].shape[2] // 2
    for j in range(num_classes-2):
        mask_class = mask_dataset[i] == j+2
        middle = (mask_class * image_dataset[i])[:,:,slice]
        middle_plus = (mask_class * image_dataset[i])[:,:,slice+1]
        middle_minus = (mask_class * image_dataset[i])[:,:,slice-1]
        mean = (middle.mean() + middle_plus.mean() + middle_minus.mean()) / 3
        std = (middle.std() + middle_plus.std() + middle_minus.std()) / 3
        median = (np.median(middle) + np.median(middle_plus) + np.median(middle_minus)) / 3
        iqr = ((np.quantile(middle, 0.75) - np.quantile(middle, 0.25)) + 
               (np.quantile(middle_plus, 0.75) - np.quantile(middle_plus, 0.25)) + 
               (np.quantile(middle_minus, 0.75) - np.quantile(middle_minus, 0.25))) / 3
        stats[f'{image_names[i]}_cylinder{[j]}'] = {'mean':mean, 'std':std, 'median':median, 'iqr':iqr}

In [46]:
import pandas as pd

df = pd.DataFrame(stats).T
df

,mean,std,median,iqr
A21_02_FatFrac_2D_FAM_BH_14_cylinder[0],0.097600,2.025646,0.0,0.0
A21_02_FatFrac_2D_FAM_BH_14_cylinder[1],0.069249,1.525212,0.0,0.0
A21_02_FatFrac_2D_FAM_BH_14_cylinder[2],0.051514,1.110349,0.0,0.0
A21_02_FatFrac_2D_FAM_BH_14_cylinder[3],0.029566,0.692030,0.0,0.0
A21_02_FatFrac_2D_FAM_BH_14_cylinder[4],0.007996,0.233750,0.0,0.0
...,...,...,...,...
A21_31_FatFrac_3D_Iron_Quant_17_cylinder[0],0.096039,2.022901,0.0,0.0
A21_31_FatFrac_3D_Iron_Quant_17_cylinder[1],0.069336,1.519939,0.0,0.0
A21_31_FatFrac_3D_Iron_Quant_17_cylinder[2],0.046733,1.009345,0.0,0.0
A21_31_FatFrac_3D_Iron_Quant_17_cylinder[3],0.030014,0.708147,0.0,0.0


In [47]:
# Save DataFrame to an Excel file
df.to_excel('stats_dataframe.xlsx', index=True)  # Include index (file names)


In [58]:
import tensorflow as tf

def dice_coef(y_true, y_pred):
    total_dice = 0.0
    num_class = 0.0
    for class_idx in range(num_classes):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]
        y_true_f = tf.keras.backend.flatten(y_true_class)
        y_pred_f = tf.keras.backend.flatten(y_pred_class)
        intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
        intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
        dice = (2. * intersection) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1e-7)
        total_dice = total_dice + dice
        num_class = num_class + 1.0
    mean_dice_score = total_dice / num_class
    return mean_dice_score

def tpr(y_true, y_pred, threshold=0.5):
    total_tpr = 0
    num_class = 0
    for class_idx in range(num_classes):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]
        y_pred_pos = tf.cast(y_pred_class > threshold, tf.float32)
        y_true_pos = tf.cast(y_true_class > threshold, tf.float32)
        true_pos = tf.reduce_sum(tf.cast(tf.logical_and(y_true_pos == 1, y_pred_pos == 1), tf.float32))
        actual_pos = tf.reduce_sum(tf.cast(y_true_pos, tf.float32))
        tpr = true_pos / (actual_pos + tf.keras.backend.epsilon())
        total_tpr += tpr
        num_class += 1
    mean_tpr = total_tpr / num_class
    return mean_tpr

def fpr(y_true, y_pred, threshold=0.5):
    total_fpr = 0
    num_class = 0
    for class_idx in range(num_classes):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]
        y_pred_pos = tf.cast(y_pred_class > threshold, tf.float32)
        y_true_neg = tf.cast(y_true_class <= threshold, tf.float32)
        false_pos = tf.reduce_sum(tf.cast(tf.logical_and(y_true_neg == 1, y_pred_pos == 1), tf.float32))
        actual_neg = tf.reduce_sum(tf.cast(y_true_neg, tf.float32))
        fpr = false_pos / (actual_neg + tf.keras.backend.epsilon())
        total_fpr += fpr
        num_class += 1
    mean_fpr = total_fpr / num_class
    return mean_fpr

model = tf.keras.models.load_model('C:/Users/User/Desktop/thoracic_seg/models/multi_thoracic_unet_model_0.h5', custom_objects={'dice_coef': dice_coef, 'tpr': tpr, 'fpr': fpr})

image_directory = 'C:/Users/User/Desktop/thoracic_seg/raw_images/'
mask_directory = 'C:/Users/User/Desktop/thoracic_seg/segmentations/'

image_dataset = []
mask_dataset = []
sliced_image_dataset = []
sliced_mask_dataset = []
image_names = []
sliced_image_names = []

images = sorted(os.listdir(image_directory))
for i, image_name in enumerate(images):  
    if i > 5:
        break  
    if (image_name.split('.')[1] == 'nii'):
        image = nib.load(image_directory+image_name)
        image = np.array(image.get_fdata())
        image_dataset.append(np.array(image))
        image_names.append(image_name.split('.')[0])

masks = sorted(os.listdir(mask_directory))
for i, image_name in enumerate(masks):
    if i > 5:
        break
    if (image_name.split('.')[1] == 'nii'):
        image = nib.load(mask_directory+image_name)
        image = np.array(image.get_fdata())
        mask_dataset.append(np.array(image))

for i in range(len(image_dataset)):
    for j in range(mask_dataset[i].shape[2]):
        sliced_image_dataset.append(image_dataset[i][:,:,j])
        sliced_mask_dataset.append(mask_dataset[i][:,:,j])
        sliced_image_names.append(image_names[i] + '-' + str(j))

sliced_image_dataset = np.array(sliced_image_dataset)
sliced_mask_dataset = np.array(sliced_mask_dataset)
image_names = np.array(image_names)
sliced_image_names = np.array(sliced_image_names)

predictions = model.predict(sliced_image_dataset)


In [65]:
for i in range(len(predictions)):
    test_img = sliced_image_dataset[i]
    ground_truth = sliced_mask_dataset[i]
    predicted_img = np.argmax(predictions[i],axis=2)
    plt.figure(figsize=(16, 8))
    plt.subplot(131)
    plt.title('Testing Image')
    plt.imshow(test_img, cmap='gray')
    plt.subplot(132)
    plt.title('Testing Label')
    plt.imshow(ground_truth, cmap='jet')
    plt.subplot(133)
    plt.title('Prediction on test image')
    plt.imshow(predicted_img, cmap='jet')
    plt.close()